In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import pickle
import h5py

Loading the dataset and normalizing

In [ ]:
test_input = pd.read_pickle('DATA/test_input')
test_output = pd.read_pickle('DATA/test_output')
index = test_input.columns

# Sort in order of increasing wind speed {This is only for visualization purpose}
test_input = test_input.sort_values(by=['windspeed'])
test_input = test_input.drop_duplicates(subset=['windspeed'], keep='last')
test_output = test_output.reindex(test_input.index)

# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)
inputn = pd.DataFrame(std_scaler.transform(test_input), columns=test_input.columns) 
# inputn is still a daraframe with numeric index
outputn = test_output/10**6  #  change of units, outputn is still a daraframe with time index

Define the models to be tested

In [ ]:
include_wave = 'no'
modelID = 3
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 
duration = '24M'

# Retrive features based on the modelID
index1 = pd.core.indexes.base.Index([]) # create a blank index array
if include_wave == 'yes': 
    index1 = index1.append(index[0:3])
if modelID == 1: # Acc17
    index1 = index1.append(index[[3,4,9,10,15,16]])
if modelID == 2: # Acc38
    index1 = index1.append(index[[5,6,11,12,17,18]])
if modelID == 3: # Acc77
    index1 = index1.append(index[[7,8,13,14,19,20]])
if modelID == 4: # Acc17&38
    index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
if modelID == 5: # Acc17&38&77
    index1 = index1.append(index[3:21])

index1 = index1.append(index[21:]) # SCADA
X = inputn[index1].values
Y = outputn.values

print(X.shape)
print(Y.shape)

In [ ]:
def NLL(y, distr): 
  return -distr.log_prob(y) 

def normal_sp(params): 
  return tfp.distributions.Normal(loc=params[:,0:2], scale=1e-3 
                                  + tf.math.softplus(0.05 * params[:,2:4]))# both parameters are learnable

kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)

inputs = tf.keras.layers.Input(shape=(X.shape[1],))

hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(inputs)
hidden = tfp.layers.DenseFlipout(64,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
params = tfp.layers.DenseFlipout(4,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn)(hidden)
dist = tfp.layers.DistributionLambda(normal_sp)(params)


model = Model(inputs=inputs, outputs=dist)
model.compile(Adam(learning_rate=0.0002), loss=NLL) 

model_params = Model(inputs=inputs, outputs=params)
model.summary()

In [ ]:
ind = np.arange(1, len(test_output)-10, 10)
x_test = test_input['windspeed'][ind]
examples = X[ind]
targets = Y[ind]

# Probababilistic Model
file = h5py.File('Weights/02_PredictionModel/Model%d_IncWave%s_%s.h5' % (modelID, include_wave, duration), 'r')
weight = []
for i in range(len(file.keys())):
    weight.append(file['weight' + str(i)][:])
model.set_weights(weight)
nsim = 1000
deml = np.zeros([len(x_test), nsim])
demn = np.zeros([len(x_test), nsim])
for j in range(nsim):
    nn_output = model.predict(examples)/10
    deml[:,j] = nn_output[:,0]
    demn[:,j] = nn_output[:,1]

M_l = np.mean(deml, axis = 1)
M_n = np.mean(demn, axis = 1)
V_l = np.std(deml, axis = 1)
V_n = np.std(demn, axis = 1)

# Deterministic Model
VSnet = load_model('Weights/01_SensorPlacementTest/DNNModel%s_IncWave%s.h5' % (modelID, include_wave))
dnn_output = VSnet.predict(examples)


In [ ]:
cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9
fig, ax = plt.subplots(2, figsize=(17*cm, 8*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.05, right=.98, top=0.98, bottom=0.12, hspace = 0.1, wspace=0.15)
ax[0].plot(x_test,targets[:,0], color = '#cb181d', linewidth=1, ls= '--')
ax[1].plot(x_test,targets[:,1], color = '#cb181d', linewidth=1, ls= '--')
ax[0].plot(x_test,dnn_output[:,0], color = '#08306b', linewidth=1, ls= '-')
ax[1].plot(x_test,dnn_output[:,1], color = '#08306b', linewidth=1, ls= '-')
ax[0].fill_between(x_test,M_l+1.96*V_l,M_l-1.96*V_l, alpha=0.5, label='95% CI (+/- 1.96std)')
ax[1].fill_between(x_test,M_n+1.96*V_n,M_n-1.96*V_n, alpha=0.5, label='95% CI (+/- 1.96std)')


ax[0].legend(['Measured', 'DNN predictions', r'BNN predictions (Mean $\pm$ 1.96 Std)'], ncol=3, loc = 'upper left')
ax[0].set_ylim([0,np.max(targets)+0.15])
ax[0].set_yticklabels([])
ax[0].set_xticklabels([])
ax[0].set_ylabel('DEM$_{tl}$')   

ax[1].legend(['Measured', 'DNN predictions', r'BNN predictions (Mean $\pm$ 1.96 Std)'], ncol=3, loc = 'upper left')
ax[1].set_yticklabels([])
ax[1].set_ylim([0,np.max(targets)+0.15])
ax[1].set_ylabel('DEM$_{tn}$')
ax[1].set_xlabel('Wind speed (m/s)')
fig.savefig('Figures/03_PredictionModel/FLComparison_DNN_BNN.pdf')

In [ ]:
fig, ax = plt.subplots(2, figsize=(17*cm, 8*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.05, right=0.99, top=0.98, bottom=0.12, hspace = 0.1, wspace=0.15)
ax[0].scatter(x_test,targets[:,0], color = '#cb181d', marker='o', facecolor='white')
ax[0].scatter(x_test,dnn_output[:,0], marker = 'x', color = 'black')
ax[0].errorbar(x_test,M_l, 1.96*V_l, marker='_', linestyle='none', capsize=5, label=r'Prediction (Mean $\pm$ 1.96 $\sigma$)')
ax[1].scatter(x_test,targets[:,1], color = '#cb181d', marker='o', facecolor='white')
ax[1].scatter(x_test,dnn_output[:,1], marker = 'x', color = 'black')
ax[1].errorbar(x_test,M_n, 1.96*V_n, marker='_', linestyle='none', capsize=5, label=r'Prediction (Mean $\pm$ 1.96 $\sigma$)')


ax[0].legend(['Measured', 'DNN predictions', r'BNN predictions (Mean $\pm$ 1.96 SD)'], ncol=3, loc = 'upper left')
ax[0].set_ylim([0,np.max(targets)+0.15])
ax[0].set_yticklabels([])
ax[0].set_xticklabels([])
ax[0].set_ylabel('DEM$_{tl}$')   

ax[1].legend(['Measured', 'DNN predictions', r'BNN predictions (Mean $\pm$ 1.96 SD)'], ncol=3, loc = 'upper left')
ax[1].set_yticklabels([])
ax[1].set_ylim([0,np.max(targets)+0.15])
ax[1].set_ylabel('DEM$_{tn}$')
ax[1].set_xlabel('Wind speed (m/s)')
fig.savefig('Figures/03_PredictionModel/FLComparison_DNN_BNN1.pdf')